# Homework 1: Text Mining
## Part 2: Text analysis

Group Members: Matias Borrel, Pol Garcia, Marvin Ernst

#### Importing relevant Libraries:

In [260]:
import os
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer
import unidecode
from textblob import TextBlob  # For sentiment analysis - we have seen this in our statistics course already, when we had to analyze hotel reviews for homework 3 last term.

[nltk_data] Downloading package stopwords to /Users/Admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/Admin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#### Define file paths:

In [261]:
files = {
    "Barcelona": {
        "During": "./Original_data/hotels_barcelona_MWC.csv",
        "Post": "./Original_data/hotels_barcelona_after_MWC.csv",
    },
    "Madrid": {
        "During": "./Original_data/hotels_madrid_MWC.csv",
        "Post": "./Original_data/hotels_madrid_after_MWC.csv",
    },
}

#### Load Data:

In [262]:
dataframes = {city: {key: pd.read_csv(path) for key, path in city_files.items()} for city, city_files in files.items()}

#### Inspect the data frames:

Print column names for each dataframe:

In [263]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        print(f"{city} - {period}:")
        print(df.columns)
        print("\n")

Barcelona - During:
Index(['Name', 'Price', 'Rating', 'Detail Link', 'stars', 'center',
       'num_coments', 'Description'],
      dtype='object')


Barcelona - Post:
Index(['Name', 'Price', 'Rating', 'Detail Link', 'stars', 'center',
       'num_coments', 'Description'],
      dtype='object')


Madrid - During:
Index(['Name', 'Price', 'Rating', 'Detail Link', 'stars', 'center',
       'num_coments', 'Description'],
      dtype='object')


Madrid - Post:
Index(['Name', 'Price', 'Rating', 'Detail Link', 'stars', 'center',
       'num_coments', 'Description'],
      dtype='object')




In [264]:
for city, periods in dataframes.items():
    print(f"{city} - During dataset shape: {periods['During'].shape}")
    print(f"{city} - Post dataset shape: {periods['Post'].shape}")

Barcelona - During dataset shape: (796, 8)
Barcelona - Post dataset shape: (999, 8)
Madrid - During dataset shape: (649, 8)
Madrid - Post dataset shape: (976, 8)


#### Preparing the Dataframes

We want that during and post treatment we have the same units (here "hotels") in each of the groups (here "city"). 

Filter Out Rows with Empty Names, or missing values for any of the columns:

In [265]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        # Drop rows with missing values in any column
        dataframes[city][period] = df.dropna(how='any')

Retain Only Names Present in Both During and Post:

In [266]:
for city, periods in dataframes.items():
    # Get the intersection of names in During and Post datasets
    common_names = set(periods['During']['Name']).intersection(set(periods['Post']['Name']))
    
    # Filter rows in both During and Post to only include these common names
    dataframes[city]['During'] = periods['During'][periods['During']['Name'].isin(common_names)]
    dataframes[city]['Post'] = periods['Post'][periods['Post']['Name'].isin(common_names)]

Verify Results:

In [267]:
for city, periods in dataframes.items():
    print(f"{city} - During dataset shape: {periods['During'].shape}")
    print(f"{city} - Post dataset shape: {periods['Post'].shape}")
    print(f"{city} - Common Names: {len(set(periods['During']['Name']).intersection(set(periods['Post']['Name'])))}")

Barcelona - During dataset shape: (339, 8)
Barcelona - Post dataset shape: (339, 8)
Barcelona - Common Names: 339
Madrid - During dataset shape: (0, 8)
Madrid - Post dataset shape: (0, 8)
Madrid - Common Names: 0


Check for duplicates:

In [244]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        # Check for duplicates in the 'Name' column
        duplicate_count = df['Name'].duplicated().sum()  # Count duplicates
        total_names = df['Name'].shape[0]  # Total number of rows
        print(f"{city} - {period}:")
        print(f"Total Names: {total_names}, Duplicates: {duplicate_count}")
        
        if duplicate_count > 0:
            # Display duplicate rows
            duplicates = df[df['Name'].duplicated(keep=False)]
            print("Duplicate Names:")
            print(duplicates[['Name']])
        print("\n")

Barcelona - During:
Total Names: 339, Duplicates: 0


Barcelona - Post:
Total Names: 339, Duplicates: 0


Madrid - During:
Total Names: 0, Duplicates: 0


Madrid - Post:
Total Names: 0, Duplicates: 0




Since we do not have any dublicates, we don't need to further filter some observations, i.e. the names uniquely identify each hotel.

#### Convert the Price to numeric:

In [245]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        # Remove all non-numeric characters (e.g., € and spaces) and dots (thousands separators)
        df['Price'] = (
            df['Price']
            .astype(str)  # Ensure the column is treated as strings
            .str.replace(r'[^\d]', '', regex=True)  # Remove non-numeric characters
            .astype(int)  # Convert cleaned strings to integers
        )

Verify the changes:

In [246]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        print(f"--- {city} - {period} ---")
        print(df[['Name', 'Price']].head())  # Show the 'Name' and cleaned 'Price' columns
        print("\n")

--- Barcelona - During ---
                                     Name  Price
0                        Isabella's House   2142
1                         Sonder Casa Luz   4870
4  Axel TWO Barcelona 4 Sup - Adults Only   2149
6                           Acta Voraport   1540
7                         Tembo Barcelona   2840


--- Barcelona - Post ---
                            Name  Price
0        chic&basic Habana Hoose   1075
3                   Hotel Market    830
6  Travelodge Barcelona Poblenou    656
7                      Exe Mitre    744
8                  Hotel Condado    698


--- Madrid - During ---
Empty DataFrame
Columns: [Name, Price]
Index: []


--- Madrid - Post ---
Empty DataFrame
Columns: [Name, Price]
Index: []




##### Deleting the Link column:

In [247]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        # Drop the 'Detail Link' column if it exists
        if 'Detail Link' in df.columns:
            df.drop(columns=['Detail Link'], inplace=True)

#### Convert the Ranking to numeric:

In [248]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        if 'Rating' in df.columns:
            # Extract numeric values, replace commas with dots, and convert to float
            df['Rating'] = (
                df['Rating']
                .astype(str)  # Ensure all values are strings
                .str.extract(r'(\d+,\d+)')  # Extract only the first valid number
                .replace(',', '.', regex=True)  # Replace comma with dot
                .astype(float)  # Convert to float
            )

Verify the changes:

In [249]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        print(f"--- {city} - {period} ---")
        print(df[['Name', 'Rating']].head())  
        print("\n")

--- Barcelona - During ---
                                     Name  Rating
0                        Isabella's House    8.58
1                         Sonder Casa Luz    8.48
4  Axel TWO Barcelona 4 Sup - Adults Only    8.38
6                           Acta Voraport    8.88
7                         Tembo Barcelona    8.88


--- Barcelona - Post ---
                            Name  Rating
0        chic&basic Habana Hoose    8.58
3                   Hotel Market    7.67
6  Travelodge Barcelona Poblenou    7.37
7                      Exe Mitre    8.38
8                  Hotel Condado    8.58


--- Madrid - During ---
Empty DataFrame
Columns: [Name, Rating]
Index: []


--- Madrid - Post ---
Empty DataFrame
Columns: [Name, Rating]
Index: []




#### Summarize and Compare:

In [250]:
for city, periods in dataframes.items():
    for period, df in periods.items():
        avg_price = round(df['Price'].mean(), 2)  # Round average price to 2 decimal places
        avg_score = round(df['Rating'].mean(), 2)  # Round average score to 2 decimal places
        print(f"{city} - {period}: Avg Price = {avg_price}, Avg Score = {avg_score}")

Barcelona - During: Avg Price = 2701.09, Avg Score = 8.36
Barcelona - Post: Avg Price = 1179.2, Avg Score = 8.37
Madrid - During: Avg Price = nan, Avg Score = nan
Madrid - Post: Avg Price = nan, Avg Score = nan


As expected, the rankings are the same, and we see changes in the price.

## (a) Preprocess the Text 

##### Looking at the first few descriptions:

In [251]:
pd.set_option('display.max_colwidth', None) # to see the full descriptions

for city, periods in dataframes.items():
    for period, df in periods.items():
        print(f"--- {city} - {period} ---")
        print(df[['Name', 'Description']].head())  
        print("\n")

--- Barcelona - During ---
                                     Name  \
0                        Isabella's House   
1                         Sonder Casa Luz   
4  Axel TWO Barcelona 4 Sup - Adults Only   
6                           Acta Voraport   
7                         Tembo Barcelona   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

##### Check whether descriptions match for both observation times:

In [252]:
for city, periods in dataframes.items():
    # Merge the During and During dataframes on 'Name'
    merged_df = periods['During'].merge(periods['Post'], on='Name', suffixes=('_During', '_Post'))
    
    mismatched = merged_df[merged_df['Description_During'] != merged_df['Description_Post']]
    
    print(f"--- {city}: Description Differences ---")
    if mismatched.empty:
        print("All descriptions match!")
    else:
        print(mismatched[['Name', 'Description_During', 'Description_Post']])
    print("\n")

--- Barcelona: Description Differences ---
                Name  \
325  Sagrada Familia   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Description_During  \
325  Sagrada Familia se encuentra en el barrio de Eixample, en Barcelona, cerca de Sagrada Familia. Dispone de terraza, wifi gratis y lavadora. Este apartamento está a 2,9 km de Platja de la Nova Icària y a 19 min a pie de Estación de metro Passeig de Gràcia.\n\nEste apartamento con aire acondicionado se compone de 1 dormitori

#### Generate a dataframe for all four observations:

In [253]:
final_data = []

for city, periods in dataframes.items():
    merged_df = periods['During'].merge(periods['Post'], on='Name', suffixes=('_During', '_Post'))
    
    merged_df['Barcelona'] = 1 if city == "Barcelona" else 0

    # During_Treatment = 1 for "During" period (event happening), 0 for "Post" period (no treatment)
    selected_columns_during = merged_df[['Name', 'Barcelona', 'Price_During', 'Rating_During', 'Description_During']].copy()
    selected_columns_during['During_Treatment'] = 1  

    selected_columns_post = merged_df[['Name', 'Barcelona', 'Price_Post', 'Rating_During', 'Description_During']].copy()
    selected_columns_post['During_Treatment'] = 0  

    selected_columns_during.rename(columns={'Price_During': 'Price', 'Rating_During': 'Rating', 'Description_During': 'Description'}, inplace=True)
    selected_columns_post.rename(columns={'Price_Post': 'Price', 'Rating_During': 'Rating', 'Description_During': 'Description'}, inplace=True)

    final_data.append(selected_columns_during)
    final_data.append(selected_columns_post)

final_df = pd.concat(final_data, ignore_index=True)

#### Normalize and Remove Special Characters:

In [254]:
final_df['Description'] = final_df['Description'].str.replace(r'[√©√±]', '', regex=True)

#### Tokenize, Remove Stop Words, and Stem:

We use Spanish Snowball Stemmer for better accuracy:

In [255]:
stemmer = SnowballStemmer("spanish")

Spanish stopwords and city names to remove:

In [256]:
stop_words = set(stopwords.words("spanish")) | {"barcelona", "madrid", "hotel"}

Regex pattern to detect numbers followed by distance or time units:

In [257]:
distance_pattern = re.compile(r"(\d+)\s+(km|kilometros|metros|millas|yardas|m|ft|feet|yd)", re.IGNORECASE)
time_pattern = re.compile(r"(\d+)\s+(minuto|minutos|hora|horas)", re.IGNORECASE)

We believe lemmatization (instead of stemming) for better Spanish text processing.

In [258]:
def preprocess_text(text):
    if pd.isna(text):
        return ""  # Handle NaN values

    text = text.lower()  # Convert to lowercase

    # Preserve numbers with distance/time expressions as a single token
    text = distance_pattern.sub(lambda x: f"{x.group(1)}_{x.group(2)}", text)
    text = time_pattern.sub(lambda x: f"{x.group(1)}_{x.group(2)}", text)

    # Remove accents, punctuation, and standalone numbers (except for preserved ones)
    text = unidecode.unidecode(text)  # Remove accents (á -> a, ñ -> n)
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\b\d+\b", "", text)  # Remove standalone numbers

    # Tokenization and Stemming
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    return " ".join(stemmed_tokens)

# Apply preprocessing to the dataset
final_df['Cleaned_Description'] = final_df['Description'].astype(str).apply(preprocess_text)

## (b) Create Word Clouds:

In [259]:
def generate_wordcloud(text, title, filename):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.savefig(filename, format='png', dpi=300, bbox_inches='tight')  # Save the word cloud
    plt.close()

# Generate and save word clouds for each city before and after preprocessing
for city_label, city_name in [(1, "Barcelona"), (0, "Madrid")]:
    city_df = final_df[final_df['Barcelona'] == city_label]
    
    # Raw descriptions
    generate_wordcloud(city_df['Description'].dropna(), f"{city_name} - Before Preprocessing", f"{city_name.lower()}_before.png")

    # Cleaned descriptions
    generate_wordcloud(city_df['Cleaned_Description'].dropna(), f"{city_name} - After Preprocessing", f"{city_name.lower()}_after.png")

ValueError: We need at least 1 word to plot a word cloud, got 0.

We can see that the words have improved a lot and now show words that seem to be more important for an analysis such as "air acondicion" or "wifi" compared to just the name of the city or "de la".

## Extracting Metadata:

In [ ]:
special_words = ["centro", "playa", "aeropuerto", "wifi", "metro", "piscina", "habitacion"]

def extract_metadata(text):
    if pd.isna(text) or text.strip() == "":
        return pd.Series([0, 0, 0, 0, 0])  # Handle empty cases

    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    
    # Compute metadata features
    word_count = len(words)
    sentence_count = len(sentences)
    avg_word_length = sum(len(word) for word in words) / max(1, word_count)  # Avoid division by zero
    sentiment_score = TextBlob(text).sentiment.polarity  # Sentiment polarity

    # Count occurrences of special words
    special_mentions = sum(text.count(word) for word in special_words)

    return pd.Series([word_count, sentence_count, avg_word_length, special_mentions, sentiment_score])

# Apply function to create metadata columns
final_df[["Word_Count", "Sentence_Count", "Avg_Word_Length", "Special_Mentions", "Sentiment_Score"]] = \
    final_df["Cleaned_Description"].apply(extract_metadata)

####  Save Cleaned Data:

In [190]:
final_df.to_csv("final_cleaned_data.csv", index=False)